# IEORE4572 MongoDB Assignment

<h3>Yuzhou Liu: yl3402</h3>
<h3>Jiayi Zheng: jz2767</h3>

In [2]:
# Google_sector_report function (Modified)
# Add date, sector name and move of the sector, biggest gainer and loser to the result
# Return a dictionary for MongoDB insertion instead of json

def google_sector_report():
    results = {}
    import requests
    import datetime
    from bs4 import BeautifulSoup
    date = datetime.datetime.today()
    if date.weekday() == 5:
        date = date - datetime.timedelta(days=1)
    if date.weekday() ==6:
        date = date - datetime.timedelta(days=2)
    url = 'https://www.google.com/finance'
    response = requests.get(url)
    if response.status_code == 200:
        google_finance = BeautifulSoup(response.content,'lxml')
        sector_table = google_finance.find_all('div', {'id': 'secperf'})
        for sectors in sector_table:
            sectorContent = sectors.find_all('tr')
            for sector in sectorContent:
                if sector.get('class') != ['colHeader']:
                    if sector.get('style') != 'height:0.9em':
                        sector_name = sector.find('a').get_text()
                        sector_link = sector.find('a').get('href')
                        sector_change_ = sector.find('span').get_text()
                        sector_change = float(sector_change_.split('%')[0])
                        sector_response = requests.get('https://www.google.com' + sector_link)
                        if sector_response.status_code == 200:
                            sector_soup = BeautifulSoup(sector_response.content,'lxml')
                            top_movers = sector_soup.find('table', {'class': 'topmovers'})
                            # if the following holds, there are both gainer and loser data.
                            if len(top_movers.find_all('tr')) == 18:
                                biggest_gainer = top_movers.find_all('tr')[1]
                                gainer_name = biggest_gainer.find('a').get_text()
                                gainer_change_ = biggest_gainer.find_all('span')[1].get_text()
                                gainer_change = float(gainer_change_.split('(')[1].split('%')[0])
                                biggest_loser = top_movers.find_all('tr')[7]
                                loser_name = biggest_loser.find('a').get_text()
                                loser_change_ = biggest_loser.find_all('span')[1].get_text()
                                loser_change = float(loser_change_.split('(')[1].split('%')[0])
                                gainer_move = abs(gainer_change)
                                loser_move = abs(loser_change)
                            # if the following holds, there is either gainer or loser data.
                            elif len(top_movers.find_all('tr')) == 12:
                                # if following holds, there is only gainer data.
                                if top_movers.find_all('tr')[0].get('class') != ['beginsec']:
                                    biggest_gainer = top_movers.find_all('tr')[1]
                                    gainer_name = biggest_gainer.find('a').get_text()
                                    gainer_change_ = biggest_gainer.find_all('span')[1].get_text()
                                    gainer_change = float(gainer_change_.split('(')[1].split('%')[0])
                                    loser_name = ''
                                    loser_change = None
                                    gainer_move = abs(gainer_change)
                                    loser_move = None
                                # else, there is only loser data.
                                else:
                                    gainer_name = ''
                                    gainer_change = None
                                    biggest_loser = top_movers.find_all('tr')[1]
                                    loser_name = biggest_loser.find('a').get_text()
                                    loser_change_ = biggest_loser.find_all('span')[1].get_text()
                                    loser_change = float(loser_change_.split('(')[1].split('%')[0])
                                    gainer_move = None
                                    loser_move = abs(loser_change)
                            # else, there is no data for gainer or loser.
                            else:
                                gainer_name = ''
                                gainer_change = None
                                loser_name = ''
                                loser_change = None
                                gainer_move = None
                                loser_move = None
                            # "..." in sector name
                            if sector_name == "Cyclical Cons. Goods ...":
                                sector_name = "Cyclical Consumer Goods & Services"
                            if sector_name == "Non-Cyclical Cons. Goods...":
                                sector_name = "Non-Cyclical Consumer Goods & Services"
                            if sector_name == "Telecommunications Servi...":
                                sector_name = "Telecommunications Services"
                            
                            results[sector_name] = {'sector_name': sector_name, 'date': str(date)[0:10],
                                                    'change': sector_change, 'move': abs(sector_change), 
                                                    'biggest_gainer': {'equity': gainer_name, 
                                                                       'change': gainer_change, 'move': gainer_move}, 
                                                    'biggest_loser': {'equity': loser_name, 
                                                                      'change': loser_change, 'move':loser_move}}
    return results

In [3]:
# Create datebase and colletion in MongoDB

import pymongo
from pymongo import MongoClient

client = MongoClient()
db = client['Google-Finance']
collection = db['sector']

In [26]:
# Create compound indexes

from pymongo import IndexModel, ASCENDING, DESCENDING
index = IndexModel([("sector_name",ASCENDING),("date", ASCENDING)],name="date_sector")

db.sector.create_indexes([index])

['date_sector']

In [10]:
# Function to update the collection as new data arrives
# The document design: Each document represents a date-sector combination.

def update_data_today():
    data = google_sector_report()
    for key in data:
        d = data[key]
        sector = d["sector_name"]
        date = d["date"]
        cursor = db.sector.find({"sector_name": sector, "date": date}).limit(1)
        if len([i for i in cursor]) != 0:
            print("Document already exists: ", date, sector)
        else:
            db.sector.insert(d)

In [11]:
# Insert data of today
update_data_today()

F:\Program Files (x86)\Anaconda3\lib\site-packages\ipykernel\__main__.py:14: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


<h4>Design an interface object for the collection. The interface object is able to answer questions:<br>
<br>
<li>1. Which sector has had the greatest move on a given date
<li>2. Which sector has had the greatest move ever
<li>3. Which stock has had the greatest move on a given date
<li>4. Which stock has had the greatest move ever
<li>5. Return a list of (date, change) tuples for a given sector
<li>6. Return a list of (sector, change) tuples for a given date
<li>7. Return the average of changes for a given sector
<li>8. Return the variance of changes for a given sector:
<li>9. Return the average changes of the biggest ganiers and losers for a given sector</h4>

In [45]:
import pymongo
from pymongo import MongoClient
client = MongoClient()

In [3]:
class Sector(MongoClient):
    def __init__(self):
        self._settings = {"db":"Google-Finance","collection":"sector"}
        self._collection = MongoClient()[self._settings["db"]][self._settings["collection"]]
        
    def p1(self, date):
        """Which sector has had the greatest move on a given date"""
        p1 = self._collection.aggregate(
            [
                {"$match": {"date" : date}},
                {"$sort": {"move": -1}}
            ])
        for i in p1:
            print("{} sector has had the greatest move on {}".format(i['sector_name'],date))
            break

    def p2(self):
        '''Which sector has had the greatest move ever'''
        p2 = self._collection.aggregate(
            [
                {"$sort": {"move": -1}}
            ])
        for i in p2:
            print("{} sector has had the greatest move ever".format(i['sector_name']))
            break
            
    def p3(self, date):
        '''Which stock has had the greatest move on a given date'''
        p31 = self._collection.aggregate(
            [
                {"$match": {"date" : date}},
                {"$sort": {"biggest_loser.move": -1}}
            ])
        p32 = self._collection.aggregate(
            [
                {"$match": {"date" : date}},
                {"$sort": {"biggest_gainer.move": -1}}
            ])
        for i in p31:
            for j in p32:
                if i['biggest_loser']['move'] > j['biggest_gainer']['move']:
                    print("{} has had the greatest move on {}".format(i['biggest_loser']['equity'],date))
                else:
                    print("{} has had the greatest move on {}".format(j['biggest_gainer']['equity'],date))
                break
            break            

    def p4(self):
        '''Which stock has had the greatest move ever'''
        p41 = self._collection.aggregate(
            [
                {"$sort": {"biggest_loser.move": -1}}
            ])
        p42 = self._collection.aggregate(
            [
                {"$sort": {"biggest_gainer.move": -1}}
            ])
        for i in p41:
            for j in p42:
                if i['biggest_loser']['move'] > j['biggest_gainer']['move']:
                    print("{} has had the greatest move ever".format(i['biggest_loser']['equity']))
                else:
                    print("{} sector has had the greatest move ever".format(j['biggest_gainer']['equity']))
                break
            break          
            
    def p5(self, sector):
        '''Return a list of (date, change) tuples for a given sector'''
        p5 = self._collection.find({"sector_name": sector})
        list5 = []
        for i in p5:
            list5.append((i['date'], i['change']))
        return list5

    def p6(self, date):
        '''Return a list of (sector, change) tuples for a given date'''
        p6 = self._collection.find({"date": date})
        list6 = []
        for i in p6:
            list6.append((i['sector_name'], i['change']))
        return list6    

    def p7(self, sector):
        '''Return the average of changes for a given sector'''
        p7 = self._collection.find({"sector_name": sector})
        list7 = []
        for i in p7:
            list7.append(i['change'])
        avg = sum(list7)/len(list7)
        print("The average change for {} is {}".format(sector, avg))
        return avg
    
    def p8(self, sector):
        '''Return the variance of changes for a given sector'''
        import numpy as np
        p8 = self._collection.find({"sector_name": sector})
        list8 = []
        for i in p8:
            list8.append(i['change'])
        var = np.var(list8)
        print("The variance of changes for {} is {}".format(sector,var))
        return var
    
    def p9(self): 
        '''Print the average changes of the biggest ganiers and losers for all sectors'''
        p9 = self._collection.aggregate(
            [
               {"$group":{"_id":"$sector_name", "gainer_avg":{"$avg": "$biggest_gainer.change"}, "loser_avg":{"$avg": "$biggest_loser.change"}}}     
            ])
        for i in p9:
            print(i)

In [4]:
sector = Sector()

In [5]:
sector.p1("2016-10-17")

Cyclical Consumer Goods & Services sector has had the greatest move on 2016-10-17


In [6]:
sector.p2()

Healthcare sector has had the greatest move ever


In [7]:
sector.p3("2016-10-14")

LML Payment Systems, Inc. has had the greatest move on 2016-10-14


In [8]:
sector.p4()

LML Payment Systems, Inc. sector has had the greatest move ever


In [9]:
sector.p5("Energy")

[('2016-10-14', -0.11), ('2016-10-17', -0.41)]

In [10]:
sector.p6("2016-10-14")

[('Utilities', -0.42),
 ('Telecommunications Services', 0.41),
 ('Financials', 0.29),
 ('Non-Cyclical Consumer Goods & Services', 0.33),
 ('Industrials', 0.13),
 ('Healthcare', -0.77),
 ('Energy', -0.11),
 ('Basic Materials', -0.07),
 ('Technology', 0.19),
 ('Cyclical Consumer Goods & Services', -0.03)]

In [11]:
sector.p7("Cyclical Consumer Goods & Services")

The average change for Cyclical Consumer Goods & Services is -0.31


-0.31

In [12]:
sector.p8("Healthcare")

The variance of changes for Healthcare is 0.087025


0.087025000000000005

In [13]:
sector.p9()

{'_id': 'Utilities', 'gainer_avg': 3.01, 'loser_avg': -2.505}
{'_id': 'Telecommunications Services', 'gainer_avg': 3.26, 'loser_avg': -3.465}
{'_id': 'Financials', 'gainer_avg': 9.37, 'loser_avg': -24.19}
{'_id': 'Non-Cyclical Consumer Goods & Services', 'gainer_avg': None, 'loser_avg': -5.38}
{'_id': 'Basic Materials', 'gainer_avg': 7.835, 'loser_avg': -4.73}
{'_id': 'Technology', 'gainer_avg': 6.715, 'loser_avg': -7.91}
{'_id': 'Industrials', 'gainer_avg': 633.43, 'loser_avg': -11.785}
{'_id': 'Energy', 'gainer_avg': 5.51, 'loser_avg': -6.15}
{'_id': 'Healthcare', 'gainer_avg': 9.345, 'loser_avg': -11.415}
{'_id': 'Cyclical Consumer Goods & Services', 'gainer_avg': 10.04, 'loser_avg': -6.390000000000001}


In [29]:
class sector_date(Sector):
    def __init__(self,date,sector):
        super().__init__()
        dbSector = self._collection.find_one({"sector_name": sector, "date": date})
        self._id = dbSector["_id"]
        self.date = dbSector['date']
        self.sector = dbSector['sector_name']
        self.change = dbSector['change']
        self.move = dbSector['move']
        self.gainer = dbSector['biggest_gainer']
        self.loser = dbSector['biggest_loser']

    def __str__(self):
        return '''
Sector: {}
Date: {}
Change: {}
The biggest gainer is {} with {}% change
The biggest loser is {} with {}% change'''.format(self.sector, self.date, self.get_change(), 
                                                          self.get_biggest_gainer(), self.get_biggest_gainer_change(),
                                                          self.get_biggest_loser(), self.get_biggest_loser_change())
        
    def get_change(self):
        return self.change
    
    def get_move(self):
        return self.move
    
    def get_biggest_gainer(self):
        return self.gainer['equity']
    
    def get_biggest_gainer_change(self):
        return self.gainer['change']
    
    def get_biggest_gainer_move(self):
        return self.gainer['move']
    
    def get_biggest_loser(self):
        return self.loser['equity']
    
    def get_biggest_loser_change(self):
        return self.loser['change']
    
    def get_biggest_loser_move(self):
        return self.loser['move']

In [30]:
energy_20161017 = sector_date("2016-10-17","Energy")

In [31]:
energy_20161017.get_change()

-0.41

In [32]:
energy_20161017.get_biggest_gainer()

'Chesapeake Granite Wash'

In [33]:
print(energy_20161017)


Sector: Energy
Date: 2016-10-17
Change: -0.41
The biggest gainer is Chesapeake Granite Wash with 3.77% change
The biggest loser is Tidewater Inc. with -6.51% change


<h4>Use functions to query:</h4>

In [46]:
import pymongo
from pymongo import MongoClient
client = MongoClient()
db = client['Google-Finance']
collection = db['sector']

def query():
    print('''
    1. Which sector has had the greatest move on a given date
    2. Which sector has had the greatest move ever
    3. Which stock has had the greatest move on a given date
    4. Which stock has had the greatest move ever
    5. Return a list of (date, change) tuples for a given sector
    6. Return a list of (sector, change) tuples for a given date
    7. Return the average of changes for a given sector
    8. Return the variance of changes for a given sector:
    9. Return the average changes of the biggest ganiers and losers for a given sector''')
    x = input("Please choose a number from 1-9: ")
    try:
        func = 'p'+x
        return globals()[func]()
    except:
        return "Something goes wrong"

In [47]:
query()


    1. Which sector has had the greatest move on a given date
    2. Which sector has had the greatest move ever
    3. Which stock has had the greatest move on a given date
    4. Which stock has had the greatest move ever
    5. Return a list of (date, change) tuples for a given sector
    6. Return a list of (sector, change) tuples for a given date
    7. Return the average of changes for a given sector
    8. Return the variance of changes for a given sector:
    9. Return the average changes of the biggest ganiers and losers for a given sector
Please choose a number from 1-9: 6
Input a date (yyyy-mm-dd): 2016-10-17


[('Non-Cyclical Consumer Goods & Services', -0.06),
 ('Telecommunications Services', -0.45),
 ('Basic Materials', 0.29),
 ('Healthcare', -0.18),
 ('Financials', -0.23),
 ('Cyclical Consumer Goods & Services', -0.59),
 ('Technology', -0.27),
 ('Energy', -0.41),
 ('Industrials', -0.2),
 ('Utilities', 0.38)]

In [34]:
# 1. Which sector has had the greatest move on a given date

def p1():
    date = input("Input a date (yyyy-mm-dd): ")
    p1 = db.sector.aggregate(
        [
            {"$match": {"date" : date}},
            {"$sort": {"move": -1}}
        ])
    
    for i in p1:
        print("{} sector has had the greatest move on {}".format(i['sector_name'],date))
        break

In [35]:
# 2. Which sector has had the greatest move ever

def p2():
    p2 = db.sector.aggregate(
        [
            {"$sort": {"move": -1}}
        ])

    for i in p2:
        print("{} sector has had the greatest move ever".format(i['sector_name']))
        break

In [36]:
# 3. Which stock has had the greatest move on a given date

def p3():
    date = input("Input a date (yyyy-mm-dd): ")
    p31 = db.sector.aggregate(
        [
            {"$match": {"date" : date}},
            {"$sort": {"biggest_loser.move": -1}}
        ])

    p32 = db.sector.aggregate(
        [
            {"$match": {"date" : date}},
            {"$sort": {"biggest_gainer.move": -1}}
        ])

    for i in p31:
        for j in p32:
            if i['biggest_loser']['move'] > j['biggest_gainer']['move']:
                print("{} has had the greatest move on {}".format(i['biggest_loser']['equity'],date))
            else:
                print("{} has had the greatest move on {}".format(j['biggest_gainer']['equity'],date))
            break
        break

In [37]:
# 4. Which stock has had the greatest move ever

def p4():
    p41 = db.sector.aggregate(
        [
            {"$sort": {"biggest_loser.move": -1}}
        ])

    p42 = db.sector.aggregate(
        [
            {"$sort": {"biggest_gainer.move": -1}}
        ])

    for i in p41:
        for j in p42:
            if i['biggest_loser']['move'] > j['biggest_gainer']['move']:
                print("{} has had the greatest move ever".format(i['biggest_loser']['equity']))
            else:
                print("{} sector has had the greatest move ever".format(j['biggest_gainer']['equity']))
            break
        break    

In [38]:
# 5. Return a list of (date, change) tuples for a given sector

def p5():
    sector = input("Input a sector: ")
    p5 = db.sector.find({"sector_name": sector})
    list5 = []
    for i in p5:
        list5.append((i['date'], i['change']))
    return list5

In [39]:
# 6. Return a list of (sector, change) tuples for a given date

def p6():
    date = input("Input a date (yyyy-mm-dd): ")
    p6 = db.sector.find({"date": date})
    list6 = []
    for i in p6:
        list6.append((i['sector_name'], i['change']))
    return list6

In [40]:
# 7. Return the average of changes for a given sector

def p7():
    sector = input("Input a sector: ")
    p7 = db.sector.find({"sector_name": sector})
    list7 = []
    for i in p7:
        list7.append(i['change'])
    avg = sum(list7)/len(list7)
    print("The average change for {} is {}".format(sector, avg))
    return avg

In [41]:
# 8. Return the variance of changes for a given sector:

def p8():
    sector = input("Input a sector: ")    
    import numpy as np
    p8 = db.sector.find({"sector_name": sector})
    list8 = []
    for i in p8:
        list8.append(i['change'])
    var = np.var(list8)
    print("The variance of changes for {} is {}".format(sector,var))
    return var

In [42]:
# 9. Print the average changes of the biggest ganiers and losers for all sectors

def p9(): 
    p9 = db.sector.aggregate(
        [
           {"$group":{"_id":"$sector_name", "gainer_avg":{"$avg": "$biggest_gainer.change"}, "loser_avg":{"$avg": "$biggest_loser.change"}}}     
        ])
    for i in p9:
        print(i)